In [2]:
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup as bs
#import scrapy
#from scrapy.crawler import CrawlerProcess
import requests
import re
import urllib.request
import os
import glob


In [3]:
df1 = pd.read_csv("dataset/train/mbti_1.csv")
print(df1.shape)
df1.head()

(8675, 2)


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [10]:
df2 = pd.read_csv("dataset/train/mbti_subreddit_data.csv")[['label','text']]
print(df2.shape)
df2.head()


(9299, 2)


,label,text
0,infj,The Tome of INFJ-lore (A user manual for INFJ)...
1,infj,A love letter to INFJs I’ve met a lot of peopl...
2,infj,It feels weird and sad to feel so much compass...
3,infj,Just want a quiet simple life I’ve realized th...
4,infj,Does anybody here feels like they want to dele...


In [11]:
print(df1['type'].value_counts())

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64


In [14]:
print(df2['label'].value_counts())

entj           897
istj           854
isfj           851
enfj           842
isfp           835
istp           780
intj           663
infj           567
estp           565
entp           508
esfp           482
enfp           457
estj           410
esfj           337
intp           126
infp            90
hvZJI8rerWA      1
hQg4C5l9aPk      1
BHkhIjG0DKc      1
7B7NHT8iN_4      1
5_PUKLlgTgQ      1
FZjOd_LAGHw      1
j5u32bxN7v4      1
w2rNWyVqYB4      1
CfOL8VfC7kk      1
Name: label, dtype: int64


In [34]:
#df2[df2['label']=="intp"]
df1 = df1.rename({"type":"label","posts":"text"},axis=1)
df1.head(1)

,label,text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...


In [41]:
df1.to_csv("dataset/train/mbti_1.csv")

In [40]:
for i in range(df1.shape[0]):
    df1['label'][i] = str(df1['label'][i]).lower()
df1.head(1)

,label,text
0,infj,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...


In [44]:
mbti = []
for val in df1['label'].unique():
    mbti.append(val)
print(mbti)

['infj', 'entp', 'intp', 'intj', 'entj', 'enfj', 'infp', 'enfp', 'isfp', 'istp', 'isfj', 'istj', 'estp', 'esfp', 'estj', 'esfj']


In [73]:
df2.drop(df2[df2["label"] == "intp"].index).shape

(9173, 2)

In [74]:
df2.shape


(9299, 2)

In [78]:
for val in df2["label"].unique():
    if val not in mbti:
        print(val)
        df2 = df2.drop(df2[df2["label"] == str(val)].index)
df2["label"].unique()

nan
5_PUKLlgTgQ
FZjOd_LAGHw
BHkhIjG0DKc
7B7NHT8iN_4
hvZJI8rerWA
j5u32bxN7v4
CfOL8VfC7kk
w2rNWyVqYB4
hQg4C5l9aPk


array(['infj', 'infp', 'intj', 'intp', 'istj', 'istp', 'isfp', 'isfj',
       'enfj', 'enfp', 'entj', 'entp', nan, 'estj', 'estp', 'esfp',
       'esfj'], dtype=object)

In [79]:
df1.head(1)



,label,text
0,infj,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...


In [80]:
df2.head(1)



,label,text
0,infj,The Tome of INFJ-lore (A user manual for INFJ)...


In [81]:
df = pd.concat([df1,df2],ignore_index=True)
df.head(5)


,label,text
0,infj,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,entp,'I'm finding the lack of me in these posts ver...
2,intp,'Good one _____ https://www.youtube.com/wat...
3,intj,"'Dear INTP, I enjoyed our conversation the o..."
4,entj,'You're fired.|||That's another silly misconce...


In [84]:
df.to_csv("dataset/train/train.csv")

In [83]:
df = df.sample(frac=1)

In [85]:
df['label'].value_counts()



infj    2037
infp    1922
intj    1754
intp    1430
entp    1193
enfp    1132
entj    1128
istp    1117
isfp    1106
istj    1059
enfj    1032
isfj    1017
estp     654
esfp     530
estj     449
esfj     379
Name: label, dtype: int64

In [89]:
dfval = df[0:1800]
dftrain = df[1800:]

In [90]:
dfval['label'].value_counts()

infj    217
intj    192
intp    165
infp    162
entj    128
entp    119
istp    112
isfp    110
enfp    109
isfj    102
istj     97
enfj     87
estp     64
estj     50
esfj     49
esfp     35
Name: label, dtype: int64

In [91]:
dftrain['label'].value_counts()




infj    1820
infp    1760
intj    1562
intp    1265
entp    1074
enfp    1023
istp    1005
entj    1000
isfp     996
istj     962
enfj     945
isfj     915
estp     590
esfp     495
estj     399
esfj     330
Name: label, dtype: int64

In [96]:
def proc(text):
    text = re.sub('\n','.',text)
    text = re.sub('\r','.',text)
    text = re.sub('\t','.',text)
    text = re.sub('{','.',text)
    text = re.sub('}','.',text)
    text = re.sub('\n','.',text)
    text = re.sub('.com','.',text)
    text = re.sub('html','.',text)
    text = re.sub('.com','.',text)
    text = re.sub("[/|{}»]",'.',text)
    text = re.sub("[^a-zA-z0-9 ,#.-]","",text)
    return text


In [112]:
dftrain2 = dftrain.copy()
dftrain2.head(1)



,label,text
2884,infp,"'[oops ignore, delete]|||@Noordenwind Yes, I a..."


In [115]:
dftrain2["text"] = dftrain["text"].map(lambda x: proc(str(x)))
dftrain2.head(3)

,label,text
2884,infp,"[oops ignore, delete]...Noordenwind Yes, I am ..."
11186,istp,ISTP Women Straight hotties. Im an ESTP male a...
1632,intj,"Yup, thats small talk in a nutshell....https....."


In [117]:
dftrain2.to_csv("dataset/train/processed_train.csv")

In [113]:
dftrain = dftrain.dropna()
dftrain2 = dftrain2.dropna()

In [114]:
dftrain2.shape

(16140, 2)

In [118]:
dfval = dfval.dropna()
dfval2 = dfval.copy()
dfval2["text"] = dfval["text"].map(lambda x: proc(str(x)))
dfval2.head(3)


,label,text
13751,enfj,How do I stop getting offended.reactive when p...
7753,enfp,Jeeze What a topic. I think you go on to any...
299,infj,"Bullies, as dumb and idiotic people as they ar..."


In [119]:
dfval.to_csv("dataset/train/notprocessed_val.csv")
dfval2.to_csv("dataset/train/processed_val.csv")


